## Pivoting your data

In this chapter, you will go one step further in generating personalized recommendations — you will find items that users, similar to the one you are making recommendations for, have liked.

The first step you will need to start with is formatting your data. You begin with a dataset containing users and their ratings as individual rows with the following columns:

    user: User ID
    title: Title of the movie
    rating: Rating the user gave the movie

You will need to transform the DataFrame into a user rating matrix where each row represents a user, and each column represents the movies on the platform. This will allow you to easily compare users and their preferences.

### Instructions 1/3
    - Inspect the first five rows of the user_ratings DataFrame to observe which columns would be most appropriate to pivot the data around.

In [ ]:
# Inspect the first 5 rows of user_ratings
print(user_ratings.head())

### Instructions 3/3
    - Transform the user_ratings DataFrame to a DataFrame containing ratings with one row per user and one column per movie and call it user_ratings_table

In [ ]:
# Transform the table
user_ratings_table = user_ratings.pivot(index='userId', columns='title', values='rating')
# Inspect the transformed table
print(user_ratings_table.head())

## Challenges with missing values

You may have noticed that the pivoted DataFrames you have been working with often have missing data. This is to be expected since users rarely see all movies, and most movies are not seen by everyone, resulting in gaps in the user-rating matrix.

In this exercise, you will explore another subset of the user ratings table user_ratings_subset that has missing values and observe how different approaches in dealing with missing data may impact its usability.

### Instructions 2/3
    - Fill the gaps in the user_ratings_subset with zeros.
    - Print and inspect the results.

In [ ]:
# Fill in missing values with 0
user_ratings_table_filled = user_ratings_subset.fillna(0)

# Inspect the result
print(user_ratings_table_filled)

## Compensating for incomplete data

For most datasets, the majority of users will have rated only a small number of items. As you saw in the last exercise, how you deal with users who do not have ratings for an item can greatly influence the validity of your models.

In this exercise, you will fill in missing data with information that should not bias the data that you do have.

You'll get the average score each user has given across all their ratings, and then use this average to center the users' scores around zero. Finally, you'll be able to fill in the empty values with zeros, which is now a neutral score, minimizing the impact on their overall profile, but still allowing the comparison of users.

user_ratings_table with a row per user has been loaded for you.

### Instructions
    - Find the average of the ratings given by each user in user_ratings_table and store them as avg_ratings.
    - Subtract the row averages from each row in user_ratings_table, and store it as user_ratings_table_centered.
    - Fill the empty values in the newly created user_ratings_table_centered with zeros.

In [ ]:
# Get the average rating for each user 
avg_ratings = user_ratings_table.mean(axis=1)

# Center each users ratings around 0
user_ratings_table_centered = user_ratings_table.sub(avg_ratings, axis=0)

# Fill in the missing data with 0s
user_ratings_table_normed = user_ratings_table_centered.fillna(0)

## User-based to item-based

By now you have a dataset with no empty values that is primed for use.

In the preceding video, you learned about both user-based recommendations and item-based recommendations. User-based recommendations compare amongst users, and item-based recommendations compare different items.

In other words, you could use user-based data to find similar users based on how they rated different movies, while you could use item-based data to find similar movies based on how they have been rated by the users.

In this exercise, you will switch between the two and compare their values.

user_ratings_subset, a subset of the user-based DataFrame you have been working with, has been loaded for you.

### Instructions 2/3
    - Transpose the user_ratings_subset table so that it is indexed by the movies and store the result as movie_ratings_subset.

In [ ]:
# Transpose the user_ratings_subset DataFrame
movie_ratings_subset = user_ratings_subset.T

print(movie_ratings_subset)

## Similar and different movie ratings

Some types of movies might be liked by one group of people, but hated by another. This might reflect the type of movie far more than its quality. Take, for example, horror movies — many people absolutely love them, while others hate them.

By understanding which movies were reviewed in a similar way, we can often find very similar movies.

In this exercise, you will compare movies and see whether they have received similar reviewing patterns.

The DataFrame movie_ratings_centered has been loaded with a row per movie, and the centered ratings it received as the values.

### Instructions 1/2
    - Assign the values for Star Wars: Episode IV and Star Wars: Episode V to sw_IV and sw_V.
    - Find their cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assign the arrays to variables
sw_IV = movie_ratings_centered.loc['Star Wars: Episode IV - A New Hope (1977)', :].values.reshape(1, -1)
sw_V = movie_ratings_centered.loc['Star Wars: Episode V - The Empire Strikes Back (1980)', :].values.reshape(1, -1)

# Find the similarity between two Star Wars movies
similarity_A = cosine_similarity(sw_IV, sw_V)
print(similarity_A)

### Instructions 2/2
    - Find the cosine similarity between the ratings for Jurassic Park (jurassic_park) and Pulp Fiction (pulp_fiction).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assign the arrays to variables
sw_IV = movie_ratings_centered.loc['Star Wars: Episode IV - A New Hope (1977)', :].values.reshape(1, -1)
sw_V = movie_ratings_centered.loc['Star Wars: Episode V - The Empire Strikes Back (1980)', :].values.reshape(1, -1)

# Find the similarity between two Star Wars movies
similarity_A = cosine_similarity(sw_IV, sw_V)

# Assign the arrays to variables
jurassic_park = movie_ratings_centered.loc['Jurassic Park (1993)', :].values.reshape(1, -1)
pulp_fiction = movie_ratings_centered.loc['Pulp Fiction (1994)', :].values.reshape(1, -1)

# Find the similarity between Pulp Fiction and Jurassic Park
similarity_B = cosine_similarity(jurassic_park, pulp_fiction)
print(similarity_B)

## Finding similarly liked movies

Just like you calculated the similarity between two movies, you can calculate it across all users to find the most similar movie to another based on how users have rated them.

The approach is similar to how you worked with content-based filtering.

You will find the similarity scores between all movies and then drill down on the movie of interest by isolating and sorting the column containing its similarity scores.

movie_ratings_centered has once again been loaded, containing each movie as a row, and their centered ratings stored as the values.

### Instructions
    - Calculate the similarity matrix between all movies in movie_ratings_centered and store it as similarities.
    - Wrap the similarities matrix in a DataFrame, with the indices of movie_ratings_centered as the columns and rows.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Generate the similarity matrix
similarities = cosine_similarity(movie_ratings_centered)

# Wrap the similarities in a DataFrame
cosine_similarity_df = pd.DataFrame(similarities, index=movie_ratings_centered.index, columns=movie_ratings_centered.index)

# Find the similarity values for a specific movie
cosine_similarity_series = cosine_similarity_df.loc['Star Wars: Episode IV - A New Hope (1977)']

# Sort these values highest to lowest
ordered_similarities = cosine_similarity_series.sort_values(ascending=False)

print(ordered_similarities)

## Stepping through K-nearest neighbors

You have just seen how K-nearest neighbors can be used to infer how someone might rate an item based on the wisdom of a (similar) crowd. In this exercise, you will step through this process yourself to ensure a good understanding of how it works.

To get you started, as you have generated similarity matrices many times before, that step has been done for you with the user similarity matrix wrapped in a DataFrame loaded as user_similarities.

This has each user as the rows and columns, and where they meet the corresponding similarity score.

In this exercise, you will be working with user_001's similarity scores, find their nearest neighbors, and based on the ratings those neighbors gave a movie, infer what rating user_001 might give it if they saw it.

### Instructions
    - Find the IDs of User_A's 10 nearest neighbors by extracting the top 10 users in ordered_similarities and storing them as nearest_neighbors.
    - Extract the ratings the users in nearest_neighbors gave from user_ratings_table as neighbor_ratings.
    - Calculate the average rating these users gave to the movie Apollo 13 (1995) to infer what User_A might give it if they had seen it.

In [ ]:
# Isolate the similarity scores for user_1 and sort
user_similarity_series = user_similarities.loc['user_001']
ordered_similarities = user_similarity_series.sort_values(ascending=False)

# Find the top 10 most similar users
nearest_neighbors = ordered_similarities[1:11].index

# Extract the ratings of the neighbors
neighbor_ratings = user_ratings_table.reindex(nearest_neighbors)

# Calculate the mean rating given by the users nearest neighbors
print(neighbor_ratings['Apollo 13 (1995)'].mean())

## Getting KNN data in shape

Now that you understand the ins and outs of how K-nearest neighbors works, you can leverage scikit-learn's implementation of KNN while recognizing what it is doing underneath the hood.

In the next two exercises, you will step through how to prepare your data for scikit-learn's KNN model, and then use it to make inferences about what rating a user might give a movie they haven't seen.

For consistency, you will once again be working with User_1 and the rating they would give Apollo 13 (1995) if they saw it.

The users_to_ratings DataFrame has again been loaded for you. This contains each user with its own row and each rating they gave as the values.

Similarly, user_ratings_table has been loaded, which contains the raw rating values (pre-centering and filling with zeros).

### Instructions 1/3
    - Drop the column corresponding to the movie you are predicting for (Apollo 13 (1995)) from the users_to_ratings DataFrame in place.
    - Extract the ratings for user_001 from the resulting users_to_ratings table and store them as target_user_x.

In [ ]:
# Drop the column you are trying to predict
users_to_ratings.drop("Apollo 13 (1995)", axis=1, inplace=True)

# Get the data for the user you are predicting for
target_user_x = users_to_ratings.loc[['user_001']]

### Instructions 2/3
    - Get the raw ratings for Apollo 13 (1995) from the user_ratings_table and store it as other_users_y.

In [ ]:
# Drop the column you are trying to predict
users_to_ratings.drop("Apollo 13 (1995)", axis=1, inplace=True)

# Get the data for the user you are predicting for
target_user_x = users_to_ratings.loc[["user_001"]]

# Get the target data from user_ratings_table
other_users_y = user_ratings_table["Apollo 13 (1995)"]

### Instructions 3/3
    - Select only the users from users_to_ratings that have rated the movie and store it as other_users_x.
    - Drop the rows from the other_users_y target that have not rated the movie.

In [ ]:
# Drop the column you are trying to predict
users_to_ratings.drop("Apollo 13 (1995)", axis=1, inplace=True)

# Get the data for the user you are predicting for
target_user_x = users_to_ratings.loc[["user_001"]]

# Get the target data from user_ratings_table
other_users_y = user_ratings_table["Apollo 13 (1995)"]

# Get the data for only those that have seen the movie
other_users_x = users_to_ratings[other_users_y.notnull()]

# Remove those that have not seen the movie from the target
other_users_y.dropna(inplace=True)

## KNN predictions

With the data in the correct shape from the last exercise, you can now use it to infer how user_001 feels about Apollo 13 (1995)

As a reminder, the data you prepared in the last exercise (and have been loaded into this one) are:

    - target_user_x - Centered ratings that user_001 has given to the movies they have seen.
    - other_users_x - Centered ratings for all other users and the movies they have rated excluding the movie Apollo 13.
    - other_users_y - Raw ratings that all other users have given the movie Apollo 13.

You will use other_users_x and other_users_y to fit a KNeighborsRegressor from scikit-learn and use it to predict what user_001 might have rated Apollo 13 (1995).

### Instructions 1/2
    - Import KNeighborsRegressor from scikit-learn.
    - Instantiate the regressor as user_knn with the metric specified as cosine and "k" set to 10.

In [ ]:
# Import the regressor
from sklearn.neighbors import KNeighborsRegressor

# Instantiate the user KNN model
user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=10)

### Instructions 2/2
    - Fit the user_knn regressor on the other_users_x and other_users_y data.
    - Using the trained model, predict what user_001 (whose ratings are stored in target_user_x) would have given the movie.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Instantiate the user KNN model
user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=10)

# Fit the model and predict the target user
user_knn.fit(other_users_x, other_users_y)
user_user_pred = user_knn.predict(target_user_x)

print(user_user_pred)

## Comparing item-based and user-based models

You have now looked at two different KNN approaches. The first was item-item KNN where you use the average of the
most similar movies that a user has rated to suggest a rating for a movie they haven't watched. The other approach was user-user KNN where you use the average of the ratings that the

most similar users gave the movie to suggest what rating the target user would give the movie.

Now, you will compare the two and calculate what rating user_002 would give to Forrest Gump.

The code for the user_rating_predictor model (that predicts based on what similar users gave the movie), and the movie_rating_predictor (that predicts based off of what ratings this user gave to similar movies) has been started for you.

KNeighborsRegressor has been imported for you.

### Instructions
    - Create a user-user K-nearest neighbors model called user_knn.
    - Fit the user_knn model then predict on target_user_x.
    - Similarly, fit an item-item K-nearest neighbors model called movie_knn, then predict ontarget_movie_x.

In [ ]:
# Instantiate the user KNN model
user_knn = KNeighborsRegressor()

# Fit the model and predict the target user
user_knn.fit(other_users_x, other_users_y)
user_user_pred = user_knn.predict(target_user_x)
print("The user-user model predicts {}".format(user_user_pred))

# Instantiate the user KNN model
movie_knn = KNeighborsRegressor()

# Fit the model on the movie data and predict
movie_knn.fit(other_movies_x, other_movies_y)
item_item_pred = movie_knn.predict(target_movie_x)
print("The item-item model predicts {}".format(item_item_pred))